<a href="https://colab.research.google.com/github/HaeSeon/nlp-ner/blob/main/%5Bcrf_ner%5Dpy_crf_suite_jp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-crfsuite
import nltk
import pycrfsuite
import warnings
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from itertools import chain

     |████████████████████████████████| 747kB 33.0MB/s 


**DATA**

In [2]:
train_set = 'train_set.txt'
test_set = 'test_set.txt'

In [3]:
f = open(train_set)
f.readlines()[:10]

['1960\tB-DATE\n',
 '年代\tI-DATE\n',
 'と\tO\n',
 '1970\tB-DATE\n',
 '年代\tI-DATE\n',
 'の\tO\n',
 '間\tO\n',
 'に\tO\n',
 '、\tO\n',
 'ジョエル\tB-PERSON\n']

In [4]:
def process_data(file_name):
  sentence=[]
  sentences=[]
  with open (file_name,'r') as f:
    for line in f.readlines():
      if "。" in line and len(sentence)>0:
        sentences.append(sentence)
        sentence=[]
      else:
        if(len(line.split('\t'))>1):
          word=line.split('\t')[0]
          tag=line.split('\t')[1]
          sentence.append((word, tag))

  return sentences

In [5]:
train_sents=process_data(train_set)
test_sents=process_data(test_set)

In [6]:
train_sents[0]

[('1960', 'B-DATE\n'),
 ('年代', 'I-DATE\n'),
 ('と', 'O\n'),
 ('1970', 'B-DATE\n'),
 ('年代', 'I-DATE\n'),
 ('の', 'O\n'),
 ('間', 'O\n'),
 ('に', 'O\n'),
 ('、', 'O\n'),
 ('ジョエル', 'B-PERSON\n'),
 ('・', 'I-PERSON\n'),
 ('モーゼス', 'I-PERSON\n'),
 ('は', 'O\n'),
 (' ', 'O\n'),
 ('プログラム', 'O\n'),
 ('中', 'O\n'),
 ('で', 'O\n'),
 ('積分', 'O\n'),
 ('問題', 'O\n'),
 ('で', 'O\n'),
 ('の', 'O\n'),
 ('記号', 'O\n'),
 ('的', 'O\n'),
 ('推論', 'O\n'),
 ('の', 'O\n'),
 ('パワー', 'O\n'),
 ('を', 'O\n'),
 ('示し', 'O\n'),
 ('た', 'O\n')]

**PyCRFSuite**

1. 주어진 모든 feature를 다 가지고 학습

In [7]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(), # word lower
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(), # word is upper?
        'word.istitle=%s' % word.istitle(), # word is title?
        'word.isdigit=%s' % word.isdigit(), # word is digit?
        #'postag=' + postag,
        #'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            #'-1:postag=' + postag1,
            #'-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
    if i < len(sent) - 1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            #'+1:postag=' + postag1,
            #'+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features

In [8]:
def sent2features(sent):
    return [word2features(sent, i) for  i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]
    
def sent2tokens(sent):
    return [token for token, label in sent]

In [9]:
sent2features(train_sents[0])[0]

['bias',
 'word.lower=1960',
 'word[-3:]=960',
 'word[-2:]=60',
 'word.isupper=False',
 'word.istitle=False',
 'word.isdigit=True',
 'BOS',
 '+1:word.lower=年代',
 '+1:word.istitle=False',
 '+1:word.isupper=False']

In [10]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [11]:
# 모델에 데이터를 append하여 학습할 준비를 한다. 
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [12]:
# 최소 다섯번 이상 등장한 feature만 이용
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True,
    
    # minimum frequency
    'feature.minfreq': 5
})

In [13]:
# 모델 학습
model_name = 'jp.crfsuite'
trainer.train(model_name)

In [14]:
tagger = pycrfsuite.Tagger()
tagger.open(model_name)

In [15]:
# 테스트 문장에 대 ner tagging 수행
example_sent = test_sents[0]
print(' '.join(sent2tokens(example_sent)), end='\n\n')

print("Predicted:", ', '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ', '.join(sent2labels(example_sent)))

再 発行 の 際 に は 手数料 510 円 と 預り 金 500 円 の 合計 1 , 010 円 が 必要 で ある

Predicted: O
, O
, O
, O
, O
, O
, O
, B-NUMBER
, I-NUMBER
, O
, O
, O
, B-NUMBER
, I-NUMBER
, O
, O
, B-MONEY
, I-MONEY
, I-MONEY
, I-MONEY
, O
, O
, O
, O

Correct:   O
, O
, O
, O
, O
, O
, O
, B-MONEY
, I-MONEY
, O
, O
, O
, B-MONEY
, I-MONEY
, O
, O
, B-MONEY
, I-MONEY
, I-MONEY
, I-MONEY
, O
, O
, O
, O



In [16]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb=preprocessing.LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [17]:
y_true = y_test
y_pred = []
for sent in test_sents:
    y_pred.append(tagger.tag(sent2features(sent)))

In [18]:
bio_classification_report(y_true,y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'                 precision    recall  f1-score   support\n\n    B-ARTIFACT\n       0.65      0.32      0.43       169\n    I-ARTIFACT\n       0.64      0.38      0.48       320\n        B-DATE\n       0.95      0.87      0.91       305\n        I-DATE\n       0.94      0.93      0.93       470\n       B-EVENT\n       0.68      0.33      0.44        40\n       I-EVENT\n       0.58      0.40      0.47        86\n    B-LOCATION\n       0.85      0.48      0.61       588\n    I-LOCATION\n       0.71      0.38      0.49       318\n       B-MONEY\n       0.90      0.41      0.56        22\n       I-MONEY\n       1.00      0.60      0.75        40\n      B-NUMBER\n       0.76      0.71      0.73       203\n      I-NUMBER\n       0.72      0.71      0.71       252\n             O\n       0.92      0.99      0.95     23268\nB-ORGANIZATION\n       0.68      0.33      0.44       341\nI-ORGANIZATION\n       0.54      0.40      0.46       389\n       B-OTHER\n       0.76      0.44      0.56       

**한정된 feature만 가지고 학습**


In [19]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
        ])
    else:
        features.append('EOS')
                
    return features

In [20]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [21]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [22]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [23]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True,
    
    # minimum frequency
    'feature.minfreq': 5
})

In [24]:
# 모델 학습
model_name = 'jp_lower_features.crfsuite'
trainer.train(model_name)
tagger = pycrfsuite.Tagger() # 학습된 모델을 tagger로 불러옴
tagger.open(model_name)

In [34]:
y_true = y_test
y_pred = []
for sent in test_sents:
    y_pred.append(tagger.tag(sent2features(sent)))

**다른 ner과 성능 비교 위해 평가 방식 변경**

B-LOC, I-LOC -> LOC, PER 로 평가방식 변경


[seqeval](https://pypi.org/project/seqeval/0.0.10/)

In [37]:
# bio_classification_report(y_true, y_pred)
# lb=preprocessing.LabelBinarizer()
# y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
# y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
# targets=["ARTIFACT","DATE","EVENT","LOCATION","MONEY","NUMBER","ORGANIZATION","OTHER","PERCENT","PERSON","TIME"]

!pip install seqeval
from seqeval.metrics import classification_report
classification_report(y_true,y_pred)

     |████████████████████████████████| 51kB 7.2MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=f5c32d6b633856852dbc9dc9d677eb23a2524e639ba0754f966db4972590ee09
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O
 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'               precision    recall  f1-score   support\n\n    ARTIFACT\n       0.62      0.31      0.41       170\n        DATE\n       0.89      0.82      0.85       306\n       EVENT\n       0.71      0.37      0.48        41\n    LOCATION\n       0.78      0.44      0.56       598\n       MONEY\n       0.78      0.30      0.44        23\n      NUMBER\n       0.69      0.53      0.60       213\nORGANIZATION\n       0.64      0.33      0.44       348\n       OTHER\n       0.60      0.32      0.42        84\n     PERCENT\n       0.68      0.32      0.43        41\n      PERSON\n       0.55      0.09      0.15       300\n        TIME\n       0.00      0.00      0.00         5\n\n    micro avg       0.74      0.41      0.53      2129\n    macro avg       0.63      0.35      0.44      2129\n weighted avg       0.71      0.41      0.51      2129\n'

**모델 확인**

영향력이 높은 features와 각각에 해당하는 weight를 확인한다. 

모든 feature를 이용한 모델로 평가했다. 

Ner tagging에서 중요한 정보는 앞/뒤에 등장하는 단어이다. 

In [63]:
from pycrfsuite import Tagger
tagger=Tagger()
tagger.open("./jp_lower_features.crfsuite")
debugger = tagger.info()
# weights = debugger.state_features
# location_features = {feature:weight for feature, weight in weights.items() if 'LOCATION' in feature[1]}

# for feature, weight in sorted(location_features.items(), key=lambda x:-x[1])[:50]:
#     print('{} : {}'.format(feature, weight))

AttributeError: ignored